<a href="https://colab.research.google.com/github/kennenvi/Challange-Dados/blob/main/Analisando_base_de_dados_com_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Utilizando o Spark no Google Colab

In [22]:
!pip install pyspark==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# Carregamento de Dados
---

## SparkSession

O ponto de entrada para programar o Spark com a API Dataset e DataFrame.

In [23]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master('local[*]') \
    .appName("Iniciando com Spark") \
    .config('spark.ui.port', '4050') \
    .getOrCreate()

In [24]:
spark

## Baixando dados

In [25]:
!wget 'https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip' && unzip semana-1.zip -d dados/

--2022-12-01 13:55:25--  https://caelum-online-public.s3.amazonaws.com/challenge-spark/semana-1.zip
Resolving caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)... 52.217.102.84, 52.217.224.65, 3.5.16.171, ...
Connecting to caelum-online-public.s3.amazonaws.com (caelum-online-public.s3.amazonaws.com)|52.217.102.84|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18975214 (18M) [application/zip]
Saving to: ‘semana-1.zip.1’

semana-1.zip.1      100%[===================>]  18.10M  50.0MB/s    in 0.4s    

2022-12-01 13:55:25 (50.0 MB/s) - ‘semana-1.zip.1’ saved [18975214/18975214]

Archive:  semana-1.zip
replace dados/dataset_bruto.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: dados/dataset_bruto.json  


# Iniciando análise

In [26]:
# Importando modulos e classes

import pyspark.sql.functions as f
from pyspark.sql.types import DoubleType, IntegerType, StringType

In [27]:
# Lendo dados e transformando-os em um DataFrame

dados = spark.read.json('/content/dados/dataset_bruto.json')

### Avaliando a estrutura dos dados

In [28]:
dados.printSchema()

root
 |-- anuncio: struct (nullable = true)
 |    |-- andar: long (nullable = true)
 |    |-- area_total: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- area_util: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- banheiros: array (nullable = true)
 |    |    |-- element: long (containsNull = true)
 |    |-- caracteristicas: array (nullable = true)
 |    |    |-- element: string (containsNull = true)
 |    |-- endereco: struct (nullable = true)
 |    |    |-- bairro: string (nullable = true)
 |    |    |-- cep: string (nullable = true)
 |    |    |-- cidade: string (nullable = true)
 |    |    |-- estado: string (nullable = true)
 |    |    |-- latitude: double (nullable = true)
 |    |    |-- longitude: double (nullable = true)
 |    |    |-- pais: string (nullable = true)
 |    |    |-- rua: string (nullable = true)
 |    |    |-- zona: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-

In [29]:
print(f'Quantidade de linhas = {dados.count()}, Quantidade de colunas {len(dados.columns)}')

Quantidade de linhas = 89083, Quantidade de colunas 3


### Selecionando os Subcampos da caluna anúncio

Porque como o objetivo é prever um valor para o imóvel apenas os dados da coluna anúncio serão utilizados

In [30]:
dados = dados.select('anuncio.*')

In [31]:
dados.show()

+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|     caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+--------------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|        []|     [16]|      [0]|                  []|{Centro, 20061003...|47d553e0-79f2-4a4...|    [0]|   [0]|       Usado|      Outros|  Comercial| [1]|[{260, 107, Venda...|
|    0|        []|     [14]|      [0]|                  []|{Centro, 20051040...|b6ffbae1-17f6-487...|    [0]|    []|       Usado|      Outros|  Comercial| [0]|[{260, 107, Venda...|
|    0|    [1026]|   [1026]|      [0]|                  []|{Maria da Graça, ...|1fb030a5-9e3e-4

## Convertendo colunas para tipos correspondetes

In [32]:
# Colunas com dados do tipo inteiro, porém em arrays
colunas_retirar_array = ['area_total', 'area_util', 'banheiros', 'quartos', 'suites', 'vaga']

In [33]:
# Retirando os dados de arrays e transformando-os em inteiros
for colunas in colunas_retirar_array:
    dados = dados.withColumn(colunas, dados[colunas][0].cast(IntegerType()))

In [34]:
# Coluna com arrays porém do tipo string
dados = dados.withColumn('caracteristicas', dados['caracteristicas'][0])

In [35]:
dados.show()

+-----+----------+---------+---------+-----------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|andar|area_total|area_util|banheiros|  caracteristicas|            endereco|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|
+-----+----------+---------+---------+-----------------+--------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+
|    0|      null|       16|        0|             null|{Centro, 20061003...|47d553e0-79f2-4a4...|      0|     0|       Usado|      Outros|  Comercial|   1|[{260, 107, Venda...|
|    0|      null|       14|        0|             null|{Centro, 20051040...|b6ffbae1-17f6-487...|      0|  null|       Usado|      Outros|  Comercial|   0|[{260, 107, Venda...|
|    0|      1026|     1026|        0|             null|{Maria da Graça, ...|1fb030a5-9e3e-4a1...|      0|  nu

## Filtrando a base de dados

Verificando frêquencia das colunas "tipo_anuncio", "tipo_unidade", "tipo_uso"

In [36]:
dados\
    .groupBy('tipo_anuncio')\
    .count()\
    .show()

+------------+-----+
|tipo_anuncio|count|
+------------+-----+
|       Usado|88827|
|  Lançamento|  256|
+------------+-----+



In [37]:
dados\
    .groupBy('tipo_unidade')\
    .count()\
    .show()

+------------+-----+
|tipo_unidade|count|
+------------+-----+
|      Outros|11963|
| Apartamento|66801|
|        Casa|10319|
+------------+-----+



In [38]:
dados\
    .groupBy('tipo_uso')\
    .count()\
    .show()

+-----------+-----+
|   tipo_uso|count|
+-----------+-----+
|  Comercial| 4542|
|Residencial|84541|
+-----------+-----+



Selecionando apenas as categorias com mais valores, pois estes representam uma parcela maior do conjunto de dados

In [39]:
dados = dados.where("tipo_uso = 'Residencial' and tipo_unidade = 'Apartamento' and tipo_anuncio = 'Usado'")

## Extraindo bairro e zona da coluna endereco

In [40]:
# Selecionando os campos
dados = dados.withColumn('bairro', dados['endereco.bairro'])
dados = dados.withColumn('zona', dados['endereco.zona'])

In [41]:
# Excluindo coluna endereco
dados = dados.drop('endereco')

In [42]:
dados.show()

+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|andar|area_total|area_util|banheiros|   caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|             valores|              bairro|      zona|
+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+--------------------+----------+
|    3|        43|       43|        1|          Academia|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|[{245, null, Vend...|           Paciência|Zona Oeste|
|    2|        42|       42|        1|     Churrasqueira|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|[{0, 0, Venda, 15...|           Paciência|Zona Oeste|
|    1|        41|       41|        1|      Portar

## Transformando os dados da coluna valores em novas colunas separadas

In [43]:
# Conferindo o esquema dos dados 
dados.printSchema()

root
 |-- andar: long (nullable = true)
 |-- area_total: integer (nullable = true)
 |-- area_util: integer (nullable = true)
 |-- banheiros: integer (nullable = true)
 |-- caracteristicas: string (nullable = true)
 |-- id: string (nullable = true)
 |-- quartos: integer (nullable = true)
 |-- suites: integer (nullable = true)
 |-- tipo_anuncio: string (nullable = true)
 |-- tipo_unidade: string (nullable = true)
 |-- tipo_uso: string (nullable = true)
 |-- vaga: integer (nullable = true)
 |-- valores: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- condominio: string (nullable = true)
 |    |    |-- iptu: string (nullable = true)
 |    |    |-- tipo: string (nullable = true)
 |    |    |-- valor: string (nullable = true)
 |-- bairro: string (nullable = true)
 |-- zona: string (nullable = true)



In [44]:
# Retirando Struct dos arrays
dados = dados\
    .withColumn('valores', f.explode('valores'))

In [45]:
# Selecionando colunas de valores
colunas_valores = dados.select('valores.*').columns

In [46]:
# Adicioando as subcolunas de valores no DataFrame dados
for col in colunas_valores:
    dados = dados.withColumn(col, dados[f'valores.{col}'])

In [47]:
# Excluindo valores
dados = dados.drop('valores')

In [48]:
dados.show()

+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-----+
|andar|area_total|area_util|banheiros|   caracteristicas|                  id|quartos|suites|tipo_anuncio|tipo_unidade|   tipo_uso|vaga|              bairro|      zona|condominio|iptu| tipo|valor|
+-----+----------+---------+---------+------------------+--------------------+-------+------+------------+------------+-----------+----+--------------------+----------+----------+----+-----+-----+
|    3|        43|       43|        1|          Academia|d2e3a3aa-09b5-45a...|      2|  null|       Usado| Apartamento|Residencial|   1|           Paciência|Zona Oeste|       245|null|Venda|15000|
|    2|        42|       42|        1|     Churrasqueira|085bab2c-87ad-452...|      2|  null|       Usado| Apartamento|Residencial|   1|           Paciência|Zona Oeste|         0|   0|Venda|15000|
|    1|        

## Selecionando apenas amostras cujo "tipo" seja Venda

In [49]:
dados = dados\
    .where('tipo = "Venda"')

# Salvando os dados

Salvando dados em formato parquet

In [51]:
dados.write\
        .parquet(
            'resultados/dados_processados_parquet',
            mode='overwrite'
            )

Salvando dados em formato csv

In [52]:
dados.write\
        .csv(
            'resultados/dados_processados_csv',
            mode='overwrite',
            header=True
            )

Salvando dados em formato ORC

In [53]:
dados.write.orc(
    'resultados/dados_processados_orc',
    mode='overwrite',    
)

### Comparando desempenho de leitura

In [ ]:
%%time
spark.read.parquet(
    '/content/drive/Othercomputers/Fino do Fino 2.0/Python Notebook/Challange_Data_Science/Semana01/dados_processados_parquet'
)

CPU times: user 2.67 ms, sys: 55 µs, total: 2.72 ms
Wall time: 108 ms


DataFrame[andar: bigint, area_total: int, area_util: int, banheiros: int, caracteristicas: string, id: string, quartos: int, suites: int, tipo_anuncio: string, tipo_unidade: string, tipo_uso: string, vaga: int, bairro: string, zona: string, condominio: string, iptu: string, tipo: string, valor: string]

In [ ]:
%%time
spark.read.csv(
    '/content/drive/Othercomputers/Fino do Fino 2.0/Python Notebook/Challange_Data_Science/Semana01/dados_processados_csv',
    inferSchema=True,
    header=True
)

CPU times: user 3.73 ms, sys: 1.23 ms, total: 4.96 ms
Wall time: 553 ms


DataFrame[andar: int, area_total: int, area_util: int, banheiros: int, caracteristicas: string, id: string, quartos: int, suites: int, tipo_anuncio: string, tipo_unidade: string, tipo_uso: string, vaga: int, bairro: string, zona: string, condominio: int, iptu: int, tipo: string, valor: int]

In [ ]:
%%time
spark.read.orc(
    '/content/drive/Othercomputers/Fino do Fino 2.0/Python Notebook/Challange_Data_Science/Semana01/dados_processados_orc'
)

CPU times: user 4.04 ms, sys: 0 ns, total: 4.04 ms
Wall time: 173 ms


DataFrame[andar: bigint, area_total: int, area_util: int, banheiros: int, caracteristicas: string, id: string, quartos: int, suites: int, tipo_anuncio: string, tipo_unidade: string, tipo_uso: string, vaga: int, bairro: string, zona: string, condominio: string, iptu: string, tipo: string, valor: string]

O desempenho de leitura de formatos parquet se mostrou mais rápido que a leitura de csv e de ORC